In [1]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using MLJ
using MLDataPattern
using DataFrames
using XGBoost
using MLJXGBoostInterface
using Plots
using MultivariateStats
using MLJBase
using Printf
using UMAP
using ThreadSafeDicts
using MLJ
using Printf
using CategoricalArrays

In [2]:
using JLD
phalp_cnn_rand = JLD.load("../data/phalp_cnn_rand_real.jld")["embedded_CNN_rand"]
phalp_bow_rand = JLD.load("../data/phalp_bow_rand_real.jld")["embedded_bow_rand"]
phalp_bow_esm = JLD.load("../data/phalp_bow_esm_real.jld")["phalp_bow_esm"]
phalp_cnn_esm = JLD.load("../data/phalp_cnn_esm_real.jld")["embedded_CNN_esm"]
up2type = JLD.load("../data/phalp_type.jld")["up2type"]

Dict{String, Union{Missing, String}} with 11549 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000172D062" => "VAL"
  "UPI001463ABBB" => "endolysin"
  "UPI000232F56D" => "endolysin"
  "UPI0011625D30" => "VAL"
  "UPI0009882324" => "endolysin"
  "UPI000CA1D611" => "VAL"
  "UPI0006BC2F8A" => "endolysin"
  "UPI000BBF7878" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI00025D6AED" => "endolysin"
  "UPI0010C2D3EE" => "endolysin"
  "UPI000D22144F" => "VAL"
  "UPI0018623B24" => "endolysin"
  "UPI00080F0655" => "endolysin"
  "UPI00022BD3A3" => "endolysin"
  "UPI0008093543" => "endolysin"
  "UPI001463E938" => "VAL"
  ⋮               => ⋮

In [3]:
new = filter(x -> (first(x) in collect(keys(phalp_cnn_esm))), up2type)
val = filter(x -> (last(x) == "VAL"), new)
endo = filter(x -> (last(x) == "endolysin"), new)

Dict{String, Union{Missing, String}} with 2803 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0012B4B15F" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000201BE9F" => "endolysin"
  "UPI000232F56D" => "endolysin"
  "UPI000DF0A1E1" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI000012EA4B" => "endolysin"
  "UPI000178C353" => "endolysin"
  "UPI001436E76F" => "endolysin"
  "UPI000F6BA7D8" => "endolysin"
  "UPI00001A38F6" => "endolysin"
  "UPI00001A9BAB" => "endolysin"
  "UPI001435C02C" => "endolysin"
  "UPI0010B96635" => "endolysin"
  "UPI0015F22725" => "endolysin"
  "UPI000D7DCFA8" => "endolysin"
  "UPI0010B8D78B" => "endolysin"
  "UPI000EB70E47" => "endolysin"
  ⋮               => ⋮

In [4]:
function divide_training_data(train_data, n_splits)
    data_splits = []
    split_size = length(train_data) ÷ n_splits
    for i in 1:n_splits
        start_index = (i - 1) * split_size + 1
        end_index = i * split_size
        push!(data_splits, train_data[start_index:end_index])
    end
    return data_splits
end

function stratifiedcvOHD(embeddings, lr = 0.035, k = 10)
    Keys = [key for (key, val) in new]
    key_seq = [embeddings[i] for i in Keys] # change mebeddings here
    Values = [val for (key, val) in new]

    lookupind = Dict(zip([i for i in 1:length(key_seq)], Values))

    (xtrain, ytrain), (xtest, ytest) = stratifiedobs((key_seq,Values), p = 0.9, shuffle=true)
    train_data_splits = divide_training_data([i for i in 1:length(xtrain)], 500)


    folf1 = []
    ClsV = zeros(10000)
    ClsE = zeros(10000)
    for i in 1:length(train_data_splits)
        x = [xtrain[j] for j in train_data_splits[i]]
        y = [ytrain[j] for j in train_data_splits[i]]
        
        if i == 1
            for j in x #init class vectors from random vector
                initer = rand([v for v in 1:length(x)])
                if ytrain[initer] == "VAL"
                    ClsV = x[initer]
                    deleteat!(x, initer)
                    deleteat!(y, initer)
                    break
                end
            end

            for j in x #init class vectors from random vector
                initer = rand([v for v in 1:length(x)])
                if ytrain[initer] == "endolysin"
                    ClsE = x[initer]
                    deleteat!(x, initer)
                    deleteat!(y, initer)
                    break
                end
            end
        end
        
        for j in 1:length(x) #training
            realcls = y[j]
            seq = x[j]
            cosV = cosine(seq, ClsV)
            cosE = cosine(seq, ClsE)

            if realcls == "VAL" && cosV < cosE
                ClsV = ClsV .+ (lr*(cosV) .* seq)
                ClsE = ClsE .- (lr*(cosE) .* seq)
            end
 
            if realcls == "endolysin" && cosV > cosE
                ClsV = ClsV .- (lr*(cosV) .* seq)
                ClsE = ClsE .+ (lr*(cosE) .* seq)
            end
        end

        confusion_matrix = zeros(2,2)

        for j in 1:length(xtest) #testing
            endo_sim = cosine(xtest[j], ClsE)
            val_sim = cosine(xtest[j], ClsV)

            if endo_sim > val_sim
                if ytest[j] == "endolysin"
                    confusion_matrix[1,1] += 1
                else
                    confusion_matrix[2,1] += 1
                end
            else
                if ytest[j] == "VAL"
                    confusion_matrix[2,2] += 1
                else
                    confusion_matrix[1,2] += 1
                end
            end
        end

        prec = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[1,2])
        recall = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[2,1])
        f1 = 2 * (prec*recall)/(prec+recall)
        push!(folf1, f1)
    end
    return folf1
        
end

stratifiedcvOHD (generic function with 3 methods)

In [16]:
t = [(phalp_bow_esm, "phalp_bow_esm"), (phalp_cnn_esm ,"phalp_cnn_esm"), (phalp_bow_rand, "phalp_bow_rand"), (phalp_cnn_rand ,"phalp_cnn_rand")]
for i in t
    f1list = stratifiedcvOHD(i[1])
    f1_plot = plot([j for j in 1:length(f1list)], f1list,
    xlabel="Batch", ylims = [0.0, 1.0],
    ylabel="F1-score", dpi = 300, tickfont=font(14), legend = false, guidefont=font(13))
    savefig(@sprintf("../thesis/Fig/%s_score.png", i[2]))
end


In [45]:
function stratifiedcvOHD(embeddings, lr = 0.035)
    Keys = [key for (key, val) in new]
    key_seq = [embeddings[i] for i in Keys] # change mebeddings here
    Values = [val for (key, val) in new]


    lookupind = Dict(zip([i for i in 1:length(key_seq)], Values))


    follow = []

        ClsV = zeros(10000)
        ClsE = zeros(10000)

        for j in 1:length(key_seq) #init class vectors from random vector
            initer = rand(1:length(key_seq))
            if Values[initer] == "VAL"
                ClsV = key_seq[initer]
                deleteat!(key_seq, initer)
                deleteat!(Values, initer)
                break
            end
        end

        for j in 1:length(key_seq) #init class vectors from random vector
            initer = rand(1:length(key_seq))
            if Values[initer] == "endolysin"
                ClsE = key_seq[initer]
                deleteat!(key_seq, initer)
                deleteat!(Values, initer)
                break
            end
        end

        for j in 1:length(key_seq) #training
            realcls = Values[j]
            seq = key_seq[j]
            cosV = cosine(seq, ClsV)
            cosE = cosine(seq, ClsE)
            println(cosE)
            if realcls == "VAL" && cosV < cosE
                ClsV = ClsV .+ (lr*(cosV) .* seq)
                ClsE = ClsE .- (lr*(cosE) .* seq)
            end
 
            if realcls == "endolysin" && cosV > cosE
                ClsV = ClsV .- (lr*(cosV) .* seq)
                ClsE = ClsE .+ (lr*(cosE) .* seq)
            end

            push!(follow, abs(cosV-cosE))
        end

    return follow
end

stratifiedcvOHD (generic function with 3 methods)

In [37]:
function stratifiedcvOHD(embeddings, lr = 0.035, k = 10)
    Keys = [key for (key, val) in new]
    key_seq = [embeddings[i] for i in Keys] # change mebeddings here
    Values = [val for (key, val) in new]
    println(length(Values))

    lookupind = Dict(zip([i for i in 1:length(key_seq)], Values))

    stratified_cv = StratifiedCV(;nfolds=k)
    pairs = MLJBase.train_test_pairs(stratified_cv, 1:length(Keys), Values)

    f1 = 0

    followbig = []

    big = zeros(2,2)

    for i in pairs
        train = i[1]
        test = i[2]

        ClsV = zeros(10000)
        ClsE = zeros(10000)

        for j in train #init class vectors from random vector
            initer = rand(train)
            if lookupind[initer] == "VAL"
                ClsV = key_seq[initer]
                filter!(!=(initer), train)
                break
            end
        end

        for j in train #init class vectors from random vector
            initer = rand(train)
            if lookupind[initer] == "endolysin"
                ClsE = key_seq[initer]
                filter!(!=(initer), train)
                break
            end
        end

        follow = []

        Threads.@threads for j in train #training
            realcls = lookupind[j]
            seq = key_seq[j]
            cosV = cosine(seq, ClsV)
            cosE = cosine(seq, ClsE)
            println(cos)

            if realcls == "VAL" && cosV < cosE
                ClsV = ClsV .+ (lr*(cosV) .* seq)
                ClsE = ClsE .- (lr*(cosE) .* seq)
            end
 
            if realcls == "endolysin" && cosV > cosE
                ClsV = ClsV .- (lr*(cosV) .* seq)
                ClsE = ClsE .+ (lr*(cosE) .* seq)
            end

            push!(follow, abs(cosV-cosE))
        end

        push!(followbig, follow)

        confusion_matrix = zeros(2,2)

        Threads.@threads for x in 1:length(test) #testing
            endo_sim = cosine(key_seq[test[x]], ClsE)
            val_sim = cosine(key_seq[test[x]], ClsV)

            if endo_sim > val_sim
                if Values[test[x]] == "endolysin"
                    confusion_matrix[1,1] += 1
                else
                    confusion_matrix[2,1] += 1
                end
            else
                if Values[test[x]] == "VAL"
                    confusion_matrix[2,2] += 1
                else
                    confusion_matrix[1,2] += 1
                end
            end
        end
        big = big .+ confusion_matrix
        #evaluating
        prec = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[1,2])
        recall = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[2,1])

        f1 += 2 * (prec*recall)/(prec+recall)
    end
    println(big ./ k)
    return f1/k, followbig
end

stratifiedcvOHD (generic function with 3 methods)

In [46]:
using LaTeXStrings
t = [(phalp_bow_esm, "phalp_bow_esm"), (phalp_cnn_esm ,"phalp_cnn_esm"), (phalp_bow_rand, "phalp_bow_rand"), (phalp_cnn_rand ,"phalp_cnn_rand")]

for i in t
    # Create a plot of the averages against their indices
    plt = plot(1:length(stratifiedcvOHD(i[1])), stratifiedcvOHD(i[1]), markershape=:circle, label = "",  seriestype=:scatter, markercolor=:blue, dpi = 300, size = (900, 500), tickfont=font(15), guidefontsize = 15, ylabel = L"$cos_{VAL} - cos_{endo}$", xlabel = "Query vector", bottom_margin=0.5Plots.cm, left_margin=0.5Plots.cm)

    dir = @sprintf("../thesis/Fig/%s_learning3.png", i[2])
    savefig(dir)
end

In [82]:
using LatexPrint
using LinearAlgebra
A = [0.0 0.4892 0.4922 0.4863 0.497 0.4933 0.4834 0.4888 0.4962 0.4894 0.4989 0.4924 0.4955 0.4959 0.4971 0.4904 0.4914 0.4877 0.4885 0.4899; 0.4892 0.0 0.4958 0.4963 0.4984 0.4915 0.4876 0.4888 0.4858 0.487 0.4805 0.491 0.4805 0.4925 0.4937 0.4932 0.4868 0.4881 0.4881 0.4949; 0.4922 0.4958 0.0 0.4857 0.5002 0.4997 0.4978 0.4946 0.5078 0.4844 0.4943 0.5 0.4927 0.4945 0.4969 0.497 0.4846 0.4853 0.4867 0.4939; 0.4863 0.4963 0.4857 0.0 0.4903 0.5008 0.4879 0.4989 0.4999 0.4971 0.499 0.4949 0.4894 0.5088 0.4878 0.4889 0.4921 0.4936 0.4824 0.4886; 0.497 0.4984 0.5002 0.4903 0.0 0.5007 0.497 0.509 0.4984 0.5042 0.5011 0.5012 0.4983 0.4929 0.5011 0.49 0.4966 0.4885 0.4955 0.4823; 0.4933 0.4915 0.4997 0.5008 0.5007 0.0 0.4929 0.4941 0.4975 0.4997 0.495 0.4905 0.4898 0.4944 0.5044 0.4917 0.4981 0.487 0.4912 0.4954; 0.4834 0.4876 0.4978 0.4879 0.497 0.4929 0.0 0.4976 0.4898 0.4938 0.4953 0.495 0.5031 0.4997 0.4857 0.5014 0.4904 0.4877 0.4923 0.4921; 0.4888 0.4888 0.4946 0.4989 0.509 0.4941 0.4976 0.0 0.499 0.4964 0.5043 0.4966 0.5021 0.5001 0.5029 0.5002 0.4928 0.4893 0.4891 0.5019; 0.4962 0.4858 0.5078 0.4999 0.4984 0.4975 0.4898 0.499 0.0 0.493 0.4965 0.492 0.4949 0.4983 0.4995 0.4946 0.4872 0.4973 0.4869 0.4819; 0.4894 0.487 0.4844 0.4971 0.5042 0.4997 0.4938 0.4964 0.493 0.0 0.4971 0.485 0.4931 0.4941 0.4877 0.4986 0.492 0.4973 0.4847 0.4889; 0.4989 0.4805 0.4943 0.499 0.5011 0.495 0.4953 0.5043 0.4965 0.4971 0.0 0.4981 0.5002 0.4972 0.496 0.4911 0.4939 0.4932 0.4932 0.4898; 0.4924 0.491 0.5 0.4949 0.5012 0.4905 0.495 0.4966 0.492 0.485 0.4981 0.0 0.5055 0.4969 0.4933 0.493 0.4912 0.4957 0.4949 0.4955; 0.4955 0.4805 0.4927 0.4894 0.4983 0.4898 0.5031 0.5021 0.4949 0.4931 0.5002 0.5055 0.0 0.4972 0.4932 0.4931 0.4919 0.4924 0.4864 0.489; 0.4959 0.4925 0.4945 0.5088 0.4929 0.4944 0.4997 0.5001 0.4983 0.4941 0.4972 0.4969 0.4972 0.0 0.4958 0.5009 0.5003 0.4874 0.4952 0.4854; 0.4971 0.4937 0.4969 0.4878 0.5011 0.5044 0.4857 0.5029 0.4995 0.4877 0.496 0.4933 0.4932 0.4958 0.0 0.5019 0.4963 0.4866 0.4946 0.4872; 0.4904 0.4932 0.497 0.4889 0.49 0.4917 0.5014 0.5002 0.4946 0.4986 0.4911 0.493 0.4931 0.5009 0.5019 0.0 0.4922 0.4949 0.4925 0.4929; 0.4914 0.4868 0.4846 0.4921 0.4966 0.4981 0.4904 0.4928 0.4872 0.492 0.4939 0.4912 0.4919 0.5003 0.4963 0.4922 0.0 0.4915 0.4859 0.4863; 0.4877 0.4881 0.4853 0.4936 0.4885 0.487 0.4877 0.4893 0.4973 0.4973 0.4932 0.4957 0.4924 0.4874 0.4866 0.4949 0.4915 0.0 0.486 0.4886; 0.4885 0.4881 0.4867 0.4824 0.4955 0.4912 0.4923 0.4891 0.4869 0.4847 0.4932 0.4949 0.4864 0.4952 0.4946 0.4925 0.4859 0.486 0.0 0.4942; 0.4899 0.4949 0.4939 0.4886 0.4823 0.4954 0.4921 0.5019 0.4819 0.4889 0.4898 0.4955 0.489 0.4854 0.4872 0.4929 0.4863 0.4886 0.4942 0.0]
tabular(round.(A, digits = 2), hlines=true)

\begin{tabular}{cccccccccccccccccccc}
$0.0$ & $0.49$ & $0.49$ & $0.49$ & $0.5$ & $0.49$ & $0.48$ & $0.49$ & $0.5$ & $0.49$ & $0.5$ & $0.49$ & $0.5$ & $0.5$ & $0.5$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.49$\\ \hline
$0.49$ & $0.0$ & $0.5$ & $0.5$ & $0.5$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.48$ & $0.49$ & $0.48$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.49$\\ \hline
$0.49$ & $0.5$ & $0.0$ & $0.49$ & $0.5$ & $0.5$ & $0.5$ & $0.49$ & $0.51$ & $0.48$ & $0.49$ & $0.5$ & $0.49$ & $0.49$ & $0.5$ & $0.5$ & $0.48$ & $0.49$ & $0.49$ & $0.49$\\ \hline
$0.49$ & $0.5$ & $0.49$ & $0.0$ & $0.49$ & $0.5$ & $0.49$ & $0.5$ & $0.5$ & $0.5$ & $0.5$ & $0.49$ & $0.49$ & $0.51$ & $0.49$ & $0.49$ & $0.49$ & $0.49$ & $0.48$ & $0.49$\\ \hline
$0.5$ & $0.5$ & $0.5$ & $0.49$ & $0.0$ & $0.5$ & $0.5$ & $0.51$ & $0.5$ & $0.5$ & $0.5$ & $0.5$ & $0.5$ & $0.49$ & $0.5$ & $0.49$ & $0.5$ & $0.49$ & $0.5$ & $0.48$\\ \hline
$0.49$ & $0.49$ & $0.5$ & $0.5$ & $0.5$ & $0.0$ & $0.49$ & $0.49$ &